# Ejercicio Práctico Solución de Koalas

* pip install findspark
* pip install pyspark==2.4.4
* pip install koalas
* pip install plotly
* pip install nbformat>=4.2.0
* conda install -n bigdata ipykernel --update-deps --force-reinstall
* pip install matplotlib

In [1]:
import findspark
findspark.init()

import pandas as pd
import pyspark

In [2]:
import numpy as np
import pandas as pd
import databricks.koalas as ks

### Funciones básicas

In [3]:
pser = pd.Series([1, 3, 5, np.nan, 6, 8]) 

kser = ks.Series([1, 3, 5, np.nan, 6, 8])


kser = ks.Series(pser)
kser = ks.from_pandas(pser)

In [4]:
kser.sort_index()

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

In [5]:
pdf = pd.DataFrame({'A': np.random.rand(5),
                    'B': np.random.rand(5)})


kdf = ks.from_pandas(pdf)

### Visualización de datos

In [6]:
kdf.describe()

,A,B
count,5.000000,5.000000
mean,0.461556,0.289697
std,0.310428,0.132534
min,0.080188,0.162273
25%,0.216439,0.209039
50%,0.506646,0.211039
75%,0.685551,0.422551
max,0.818953,0.443583


In [7]:
kdf.sort_values(by='B')

,A,B
2,0.216439,0.162273
4,0.818953,0.209039
1,0.506646,0.211039
3,0.080188,0.422551
0,0.685551,0.443583


In [8]:
kdf.transpose()

,0,1,2,3,4
A,0.685551,0.506646,0.216439,0.080188,0.818953
B,0.443583,0.211039,0.162273,0.422551,0.209039


### Selección

In [9]:
kdf[['A', 'B']]

,A,B
0,0.685551,0.443583
1,0.506646,0.211039
2,0.216439,0.162273
3,0.080188,0.422551
4,0.818953,0.209039


In [10]:
kdf.loc[1:2]

,A,B
1,0.506646,0.211039
2,0.216439,0.162273


In [11]:
kdf.iloc[:3, 1:2]

,B
0,0.443583
1,0.211039
2,0.162273


### Aplicando funciones de Python a Koalas

In [12]:
kdf.apply(np.cumsum)

,A,B
0,0.685551,0.443583
1,1.192197,0.654621
2,1.408637,0.816894
3,1.488825,1.239445
4,2.307778,1.448485


In [13]:
kdf.apply(lambda x: x ** 2)

,A,B
0,0.469980,0.196766
1,0.256690,0.044537
2,0.046846,0.026332
3,0.006430,0.178550
4,0.670684,0.043697


### Agrupando datos

In [14]:
kdf.groupby(['A', 'B']).sum()

,
A,B
0.685551,0.443583
0.216439,0.162273
0.818953,0.209039
0.080188,0.422551
0.506646,0.211039


### Generando gráficos

In [15]:
# This is needed for visualizing plot on notebook
%matplotlib inline

In [16]:
speed = [0.1, 17.5, 40, 48, 52, 69, 88]
lifespan = [2, 8, 70, 1.5, 25, 12, 28]
index = ['snail', 'pig', 'elephant',
         'rabbit', 'giraffe', 'coyote', 'horse']
kdf = ks.DataFrame({'speed': speed,
                   'lifespan': lifespan}, index=index)
kdf.plot.barh()

In [17]:
kdf = ks.DataFrame({
    'sales': [3, 2, 3, 9, 10, 6, 3],
    'signups': [5, 5, 6, 12, 14, 13, 9],
    'visits': [20, 42, 28, 62, 81, 50, 90],
}, index=pd.date_range(start='2019/08/15', end='2020/03/09',
                       freq='M'))
kdf.plot.area()

### Utilizando SQL en Koalas

In [18]:
kdf = ks.DataFrame({'year': [1990, 1997, 2003, 2009, 2014],
                    'pig': [20, 18, 489, 675, 1776],
                    'horse': [4, 25, 281, 600, 1900]})

In [19]:
ks.sql("SELECT * FROM {kdf} WHERE pig > 100")

,year,pig,horse
0,2003,489,281
1,2009,675,600
2,2014,1776,1900


In [20]:
pdf = pd.DataFrame({'year': [1990, 1997, 2003, 2009, 2014],
                    'sheep': [22, 50, 121, 445, 791],
                    'chicken': [250, 326, 589, 1241, 2118]})

In [21]:
ks.sql('''
    SELECT ks.pig, pd.chicken
    FROM {kdf} ks INNER JOIN {pdf} pd
    ON ks.year = pd.year
    ORDER BY ks.pig, pd.chicken''')

,pig,chicken
0,18,326
1,20,250
2,489,589
3,675,1241
4,1776,2118


### Trabajando con PySpark

In [22]:
kdf = ks.DataFrame({'A': [1, 2, 3, 4, 5], 'B': [10, 20, 30, 40, 50]})
sdf = kdf.to_spark()
type(sdf)

pyspark.sql.dataframe.DataFrame

In [23]:
sdf.show()

+---+---+
|  A|  B|
+---+---+
|  1| 10|
|  2| 20|
|  3| 30|
|  4| 40|
|  5| 50|
+---+---+

